# InfCtx trainer baseline setup
The trainer validation is mostly done using the either of the following

**1B5 model wtih**
- Layer count: 24
- Embed size: 2048

**L6-D512 model with**
- Layer count: 6
- Embed size: 512

Typically with the following dataset
- "teven/enwiki_10k" dataset, chunked to 1024 token sizes

The following notebook, helps perform the basic download and setup for the "init model", and "test dataset". Which is used as a reference point for all other validation processes (unless stated otherwise)

Generally you only need to do this once

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps
>
> All training runs (except dryrun) is configured to log to weights and bias, comment out the logger in the config file if you want to avoid this

## Preparing the init model and test dataset

In [6]:
# First lets setup the various directories, and get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/
!cd ../../model/ && wget -nc https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
!ls -alh ../../model/Echo-A-1B5-Init.pth

File ‘Echo-A-1B5-Init.pth’ already there; not retrieving.

-rw-rw-r-- 1 picocreator picocreator 2.9G Jun 22 12:41 ../../model/Echo-A-1B5-Init.pth


In [2]:
# Lets initialized the L6-D512 model with the init_model.py code
!cd ../../RWKV-v4neo/ && python3 init_model.py --n_layer 6 --n_embd 512 --vocab_size neox ../model/L6-D512-neox-init.pth

[2023-07-30 14:30:20,457] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230706'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-neox-init.pth
Vocab size: 50277
---- ----- ----
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_2048_bf16/build.ninja...
Building extension module wkv_2048_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_2048_bf16...
50277 512   -0.1 emb.weight
512   512   0    blocks.0.att.key.weight
512   512   1.0  blocks.0.att.value.weight
512   512   0    blocks.0.att.receptance.weight
512   512   0    blocks.0.att.output.weigh

In [1]:
# Lets preload the requried dataset
!cd ../../RWKV-v4neo && python3 preload_datapath.py ../notebook/trainer-validation/config/baseline-dryrun.yaml

Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 967.32it/s]


# Trainer Code validation via dryrun

The following dryrun, help do a basic check that the existing trainer code changes are valid across 2 * 2 data samples.

If this check fail, its most probably a code / envrionment setup issue (no further checks needed)

It does not log the run the W&B

In [10]:
# Validate source code and env is working, by doing a short 2 sample dryrun
!cd ../../RWKV-v4neo && python3 lightning_trainer.py fit -c ../notebook/trainer-validation/config/baseline-dryrun.yaml

[2023-07-26 00:06:34,193] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230725'
Global seed set to 3941088705
Using /home/picocreator/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu118/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_128_bf16...
/home/picocreator/anaconda3/envs/rwkv-infctx-nightly/lib/python3.11/site-packages/lightning/fabric/connector.py:554: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU ava

# (Optional) Baseline full context (1024) training

Perform a full 1 epoch training run of training context size = 1024. Ensuring all data samples fit within the allocated training size. And is used as the baseline loss comparision for several experiments

In [9]:
# Full training run
!cd ../../RWKV-v4neo && python3 lightning_trainer.py fit -c ../notebook/trainer-validation/config/baseline-1024.yaml

[2023-07-01 20:53:18,310] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230701_205320-k8flu72z
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-validation-full (train-ctx=1024, data-ctx=1024, bs=12)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/k8flu72z
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by